In [ ]:
!pip install detoxify

     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 3.8 MB 42.7 MB/s 
     |████████████████████████████████| 77 kB 2.9 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
     |████████████████████████████████| 880 kB 39.2 MB/s 
     |████████████████████████████████| 6.6 MB 25.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e13557170110dad7415432099c435a6ffedb8e79d83c01d1d13242b036c79a05
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from detoxify import Detoxify
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
data_path = '/content/drive/MyDrive/nlu_project/'
df = pd.read_csv(data_path+'labelled_master_data_2022-05-11.csv')

Mounted at /content/drive/


In [ ]:
#Bert
model = Detoxify('original')

In [ ]:
#Get BERT predictions
#Doing it in batches of 10 to not overload ram
length = df.shape[0]
inds = np.linspace(0,length, 81)
inds = [int(num) for num in inds]
df_list = []
for i in range(len(inds)-1):
  results = model.predict(list(df['comment_for_evaluation'][inds[i]:inds[i+1]].values))
  df_ind = list(df['example_id'][inds[i]:inds[i+1]].values)
  results_df = pd.DataFrame(results, index=df_ind).round(5)
  df_list.append(results_df)

In [ ]:
out = pd.concat(df_list)
out.rename(columns = {'obscene':'profanity'}, inplace=True)
# out['profanity'] = np.maximum(out['obscene'],out['sexual_explicit']) # Uncomment for Roberta
out = out.round(0)
out.head()

,toxicity,severe_toxicity,profanity,threat,insult,identity_attack
275362,0.0,0.0,0.0,0.0,0.0,0.0
30108,1.0,0.0,1.0,0.0,0.0,0.0
316461,1.0,0.0,1.0,0.0,0.0,0.0
316957,1.0,0.0,1.0,0.0,0.0,0.0
318752,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# df.drop(df.iloc[:, [0]], axis=1, inplace=True)
for column in out.columns:
  df[column] = out[column].values
df.head()

,example_id,preceding_comment,comment_for_evaluation,following_comment,toxicity,severe_toxicity,identity_attack,insult,profanity,threat,obscene
0,275362,Which one of u bought my stocks? >:(,posted this in the other thread but the more i...,That thing is 1.64% rn. JPow has to say somet...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30108,Alexa play down by 311.,No one cares about 10 year bonds bro - thats j...,Sorry you were born the bad kind of retarded.,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,316461,Today = no Vaseline,The entire market is fucking hemorrhaging,Imagine putting your money in bonds lol,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,316957,definitely. keep buying guys !!!!! HURRRDURRr,damn so my puts are fukd,"Quick, everyone buy Into the bull trap again!",1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,318752,Big if true.,Do you routinely comment your opinion on topic...,"They get all 4, fuck that. \n\nJust one Warloc...",0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.to_csv(data_path+'baseline_bert_5_11.csv', index = False)

In [ ]:
#Roberta
model = Detoxify('unbiased')

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.3-alpha/toxic_debiased-c7548aa0.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_debiased-c7548aa0.ckpt


  0%|          | 0.00/476M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
#Get RoBERTa predictions
#Doing it in batches of 10 to not overload ram
length = df.shape[0]
inds = np.linspace(0,length, 81)
inds = [int(num) for num in inds]
df_list = []
for i in range(len(inds)-1):
  results = model.predict(list(df['comment_for_evaluation'][inds[i]:inds[i+1]].values))
  df_ind = list(df['example_id'][inds[i]:inds[i+1]].values)
  results_df = pd.DataFrame(results, index=df_ind).round(5)
  df_list.append(results_df)

In [ ]:
out = pd.concat(df_list)
out['profanity'] = np.maximum(out['obscene'],out['sexual_explicit'])
out.drop(columns = ['obscene', 'sexual_explicit'], inplace = True)
out = out.round(0)
out.head()

,toxicity,severe_toxicity,identity_attack,insult,threat,profanity
275362,0.0,0.0,0.0,0.0,0.0,0.0
30108,1.0,0.0,0.0,0.0,0.0,1.0
316461,1.0,0.0,0.0,0.0,0.0,1.0
316957,1.0,0.0,0.0,1.0,0.0,1.0
318752,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for column in out.columns:
  df[column] = out[column].values
df.head()

,example_id,preceding_comment,comment_for_evaluation,following_comment,toxicity,severe_toxicity,identity_attack,insult,profanity,threat,obscene
0,275362,Which one of u bought my stocks? >:(,posted this in the other thread but the more i...,That thing is 1.64% rn. JPow has to say somet...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30108,Alexa play down by 311.,No one cares about 10 year bonds bro - thats j...,Sorry you were born the bad kind of retarded.,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,316461,Today = no Vaseline,The entire market is fucking hemorrhaging,Imagine putting your money in bonds lol,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,316957,definitely. keep buying guys !!!!! HURRRDURRr,damn so my puts are fukd,"Quick, everyone buy Into the bull trap again!",1.0,0.0,0.0,1.0,1.0,0.0,1.0
4,318752,Big if true.,Do you routinely comment your opinion on topic...,"They get all 4, fuck that. \n\nJust one Warloc...",0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.to_csv(data_path+'baseline_roberta_5_11.csv', index = False)

In [ ]:
# DeBERTA
# Training and predictions were done on GCP
# This only reads in the predictions
db_preds = pd.read_csv(data_path+'deberta_preds.csv')
db_preds.head()

,toxicity,severe_toxicity,obscene,threat,insult,identity_attack
0,0.00004,0.00000,0.00001,0.00001,0.00001,0.00001
1,0.96433,0.00228,0.53017,0.00117,0.00355,0.00121
2,0.99879,0.57718,0.99479,0.00360,0.29111,0.00084
3,0.99696,0.19042,0.99326,0.00110,0.07931,0.00051
4,0.00012,0.00001,0.00001,0.00001,0.00001,0.00001


In [ ]:
db_preds = db_preds.round(0)
for column in db_preds.columns:
  df[column] = db_preds[column].values
df.head()

,example_id,preceding_comment,comment_for_evaluation,following_comment,toxicity,severe_toxicity,identity_attack,insult,profanity,threat,obscene
0,275362,Which one of u bought my stocks? >:(,posted this in the other thread but the more i...,That thing is 1.64% rn. JPow has to say somet...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30108,Alexa play down by 311.,No one cares about 10 year bonds bro - thats j...,Sorry you were born the bad kind of retarded.,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,316461,Today = no Vaseline,The entire market is fucking hemorrhaging,Imagine putting your money in bonds lol,1.0,1.0,0.0,0.0,1.0,0.0,1.0
3,316957,definitely. keep buying guys !!!!! HURRRDURRr,damn so my puts are fukd,"Quick, everyone buy Into the bull trap again!",1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,318752,Big if true.,Do you routinely comment your opinion on topic...,"They get all 4, fuck that. \n\nJust one Warloc...",0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.to_csv(data_path+'baseline_deberta_5_11.csv', index = False)

##Evaluation

In [ ]:
bert_p = pd.read_csv(data_path+'baseline_bert_5_11.csv')
roberta_p = pd.read_csv(data_path+'baseline_roberta_5_11.csv')
deberta_p = pd.read_csv(data_path+'baseline_deberta_5_11.csv')
df = pd.read_csv(data_path+'labelled_master_data_2022-05-11.csv')

In [ ]:
df.loc[df['identity_attack'].isna()] = df.loc[df['identity_attack'].isna()].fillna(0)

In [ ]:
# Bert Over all eval 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
preds = pd.concat([bert_p['toxicity'],bert_p['severe_toxicity'], 
                           bert_p['identity_attack'], bert_p['insult'], bert_p['profanity'], bert_p['threat']])
truths = pd.concat([df['toxicity'], df['severe_toxicity'], 
                           df['identity_attack'], df['insult'], df['profanity'], df['threat']])

bert_overall = precision_recall_fscore_support(truths, preds)
bert_overall

(array([0.91648189, 0.53846154]),
 array([0.91580502, 0.54065041]),
 array([0.91614333, 0.53955375]),
 array([4062,  738]))

In [ ]:
# Roberta Over all eval 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
preds = pd.concat([roberta_p['toxicity'],roberta_p['severe_toxicity'], 
                           roberta_p['identity_attack'], roberta_p['insult'], roberta_p['profanity'], roberta_p['threat']])
truths = pd.concat([df['toxicity'], df['severe_toxicity'], 
                           df['identity_attack'], df['insult'], df['profanity'], df['threat']])

robert_overall = precision_recall_fscore_support(truths, preds)
robert_overall

(array([0.91623553, 0.53711201]),
 array([0.91555884, 0.53929539]),
 array([0.91589706, 0.53820149]),
 array([4062,  738]))

In [ ]:
# Deberta Over all eval 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
preds = pd.concat([deberta_p['toxicity'],deberta_p['severe_toxicity'], 
                           deberta_p['identity_attack'], deberta_p['insult'], deberta_p['profanity'], deberta_p['threat']])
truths = pd.concat([df['toxicity'], df['severe_toxicity'], 
                           df['identity_attack'], df['insult'], df['profanity'], df['threat']])

debert_overall = precision_recall_fscore_support(truths, preds)
debert_overall

(array([0.91182266, 0.51351351]),
 array([0.91137371, 0.51490515]),
 array([0.91159813, 0.51420839]),
 array([4062,  738]))

In [ ]:
from gc import DEBUG_COLLECTABLE
DEBUG_COLLECTABLE

In [ ]:
# Bert Column Eval
preds = bert_p[['toxicity','severe_toxicity', 'identity_attack', 'insult',
                'profanity', 'threat']]
truths = df[['toxicity','severe_toxicity', 'identity_attack', 'insult',
                'profanity', 'threat']]

bert_col = precision_recall_fscore_support(truths, preds)       
bert_col    

(array([0.22674419, 0.33333333, 1.        , 0.42372881, 0.98161765,
        0.        ]),
 array([0.71559633, 0.04166667, 0.04477612, 0.38461538, 0.65925926,
        0.        ]),
 array([0.34437086, 0.07407407, 0.08571429, 0.40322581, 0.788774  ,
        0.        ]),
 array([109,  24,  67, 130, 405,   3]))

In [ ]:
# RoBerta Column Eval
preds = roberta_p[['toxicity','severe_toxicity', 'identity_attack', 'insult',
                'profanity', 'threat']]
truths = df[['toxicity','severe_toxicity', 'identity_attack', 'insult',
                'profanity', 'threat']]

robert_col = precision_recall_fscore_support(truths, preds)       
robert_col  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.20606061, 0.        , 0.8       , 0.42857143, 0.97482014,
        0.5       ]),
 array([0.62385321, 0.        , 0.05970149, 0.41538462, 0.6691358 ,
        0.33333333]),
 array([0.30979499, 0.        , 0.11111111, 0.421875  , 0.79355783,
        0.4       ]),
 array([109,  24,  67, 130, 405,   3]))

In [ ]:
# DeBerta Column Eval
preds = deberta_p[['toxicity','severe_toxicity', 'identity_attack', 'insult',
                'profanity', 'threat']]
truths = df[['toxicity','severe_toxicity', 'identity_attack', 'insult',
                'profanity', 'threat']]

debert_col = precision_recall_fscore_support(truths, preds)       
debert_col 

(array([0.19130435, 0.11764706, 0.375     , 0.41758242, 0.97482014,
        0.        ]),
 array([0.60550459, 0.08333333, 0.04477612, 0.29230769, 0.6691358 ,
        0.        ]),
 array([0.2907489 , 0.09756098, 0.08      , 0.3438914 , 0.79355783,
        0.        ]),
 array([109,  24,  67, 130, 405,   3]))